<h1 align= "center"><b>Problem: Doomsday Fuel</b></h1>

Making fuel for the LAMBCHOP's reactor core is a tricky process because of the exotic matter involved. It starts as raw ore, then during processing, begins randomly changing between forms, eventually reaching a stable form. There may be multiple stable forms that a sample could ultimately reach, not all of which are useful as fuel. 

Commander Lambda has tasked you to help the scientists increase fuel creation efficiency by predicting the end state of a given ore sample. You have carefully studied the different structures that the ore can take and which transitions it undergoes. It appears that, while random, the probability of each structure transforming is fixed. That is, each time the ore is in 1 state, it has the same probabilities of entering the next state (which might be the same state).  You have recorded the observed transitions in a matrix. The others in the lab have hypothesized more exotic forms that the ore can become, but you haven't seen all of them.

Write a function solution(m) that takes an array of array of nonnegative ints representing how many times that state has gone to the next state and return an array of ints for each terminal state giving the exact probabilities of each terminal state, represented as the numerator for each state, then the denominator for all of them at the end and in simplest form. The matrix is at most 10 by 10. It is guaranteed that no matter which state the ore is in, there is a path from that state to a terminal state. That is, the processing will always eventually end in a stable state. The ore starts in state 0. The denominator will fit within a signed 32-bit integer during the calculation, as long as the fraction is simplified regularly. 

    For example, consider the matrix m:
    [
	    [0,1,0,0,0,1],  # s0, the initial state, goes to s1 and s5 with equal probability
	    [4,0,0,3,2,0],  # s1 can become s0, s3, or s4, but with different probabilities
	    [0,0,0,0,0,0],  # s2 is terminal, and unreachable (never observed in practice)
	    [0,0,0,0,0,0],  # s3 is terminal
	    [0,0,0,0,0,0],  # s4 is terminal
	    [0,0,0,0,0,0],  # s5 is terminal
    ]

So, we can consider different paths to terminal states, such as:

    s0 -> s1 -> s3
    s0 -> s1 -> s0 -> s1 -> s0 -> s1 -> s4
    s0 -> s1 -> s0 -> s5

Tracing the probabilities of each, we find that:

    s2 has probability 0
    s3 has probability 3/14
    s4 has probability 1/7
    s5 has probability 9/14

So, putting that together, and making a common denominator, gives an answer in the form of [s2.numerator, s3.numerator, s4.numerator, s5.numerator, denominator] which is [0, 3, 2, 9, 14].

<h3 align= "center"><b>Test Cases</b></h3>

```

Input:
    solution.solution([[0, 2, 1, 0, 0], [0, 0, 0, 3, 4], [0, 0, 0, 0, 0], [0, 0, 0, 0,0], [0, 0, 0, 0, 0]])
Output:
    [7, 6, 8, 21]

Input:
    solution.solution([[0, 1, 0, 0, 0, 1], [4, 0, 0, 3, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]])
Output:
    [0, 3, 2, 9, 14]

```

<h3 align= "center"><b>The idea</b></h3>

- First idea: You can ignore the fuel that stays in the same form. Since it will eventually go to the other form, you can just make that a 0 and it won't change the final result. So we can do:


In [ ]:
for i in range(siz):
    mat[i][i] = 0

Now the key idea: we want to transform the matrix into an "equivalent" one for which the first row (which tells us the states to which the fuel evolve from the start) only transforms the fuel into terminal states. Then that row will basically be our answer. By "equivalent" I mean that it still represents the same process by which the fuel evolves. We will get to equivalent matrices by doing allowed operations that put new zeros in the matrix, similar to how you <a href = "https://en.wikipedia.org/wiki/Row_echelon_form">reduce a matrix</a> in linear algebra. We will do this with a loop for each col and row, and in that step we will obtain a zero in mat[row][col], except if the column is a terminal state:

In [23]:
def solution(mat):

    siz = len(mat)
    
    for i in range(siz):
        mat[i][i] = 0
    
    sum_rows = [sum(row) for row in mat ]
    if sum_rows[0] == 0:
        return [1,1]
       
    def dot(num,arr):
        return [num * n for n in arr]
    def add(arr,arr2):
        return [n1 + n2 for n1,n2 in zip(arr,arr2)]
    
    for col in range(0,siz):
        if sum_rows[col] == 0:
            continue
        for row in range(0,siz):
            if mat[row][col] == 0:
                continue
            prov = add ( dot(sum_rows[col] , mat[row]) , dot(mat[row][col] , mat[col]))
            prov[col] = 0
            prov[row] = 0     
            mat[row] = prov
            sum_rows[row] = sum( prov )
            print(col, row, mat)

solution([  [0, 2, 1, 0, 0], 
            [0, 0, 0, 3, 4], 
            [0, 0, 0, 0, 0], 
            [0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0]])

1 0 [[0, 0, 7, 6, 8], [0, 0, 0, 3, 4], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]


First the selling point: Note that the row [0, 0, 7, 6, 8] in the first line of the matrix already gives us the desired answer [7,6,8,21]! This row is saying that the process will have the same outcome if the fuel goes directly from state 0 to the terminal states 2, 3, and 4, with those proportions.

Now, the key code line to understand is 

(***)  prov = add ( dot(sum_rows[col] , mat[row]) , dot(mat[row][col] , mat[col]))

which computes a new row that will replace mat[row]. This is easiest to understand in the example, so with col = 1 and row = 0. This is representing the following process:

Start with 3 * 7 = 21 fuel in state 0, 7 * 2 goes to state 1 and then it gets modified again, so 3 * 2 goes to state 3 and 4 * 2 goes to state 4. Also 7 * 1 goes directly from state 0 to state 2. The new state is then [0,0,7*1,3*2,4*2], so this could replace row 0, and you can get that by first doing the computation in (***), which in this case is:

7 * [0,2,1,0,0] + 2 * [0,0,0,3,4] = [0,14,7,6,8]

and then making the second coordinate (14) into 0 (that's the code line prov[col] = 0) because those 7*2 in state 1 were already modified into [0,0,0,6,8]. Finally, since this will go into mat[row], we can also make prov[row]=0 like we did for our "first idea". This guarantees that we keep getting new zeros and not losing the ones we already had.

And that's pretty much it! After all this process you will have a first row which will give you the answer once you divide all the numbers by their gcd.

Note: the line "if mat[row][col] == 0: continue" is not needed and it actually wasn't part of my solution, but it keeps the numbers smaller and so it makes the explanation a bit easier here...

Here's one more example with the other test case, and below is the complete solution


In [25]:
solution([[0, 1, 0, 0, 0, 1], [4, 0, 0, 3, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]])

0 1 [[0, 1, 0, 0, 0, 1], [0, 0, 0, 6, 4, 4], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
1 0 [[0, 0, 0, 6, 4, 18], [0, 0, 0, 6, 4, 4], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]


<h3 align= "center"><b>The code</b></h3>

In [8]:
def solution(mat):

    siz = len(mat)
    
    for i in range(siz):
        mat[i][i] = 0
    
    sum_rows = [sum(row) for row in mat ]
    if sum_rows[0] == 0:
        return [1,1]
       
    def dot(num,arr):
        return [num * n for n in arr]
    def add(arr,arr2):
        return [n1 + n2 for n1,n2 in zip(arr,arr2)]
    
    for col in range(0,siz):
        if sum_rows[col] == 0:
            continue
        for row in range(0,siz):
            prov = add ( dot(sum_rows[col] , mat[row]) , dot(mat[row][col] , mat[col]))
            prov[col] = 0
            prov[row] = 0     
            mat[row] = prov
            sum_rows[row] = sum( prov )
            
    ans = [mat[0][i] for i in range(siz) if sum_rows[i] == 0]
    from math import gcd
    from functools import reduce
    gcd = reduce(gcd,ans)
    ans = [a//gcd for a in ans]
    ans.append (sum(ans))
    return ans


In [9]:
print (solution([[0, 2, 1, 0, 0], [0, 0, 0, 3, 4], [0, 0, 0, 0, 0], [0, 0, 0, 0,0], [0, 0, 0, 0, 0]]))

[7, 6, 8, 21]


In [10]:
print (solution([[0, 1, 0, 0, 0, 1], [4, 0, 0, 3, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]))

[0, 3, 2, 9, 14]
